# Ensemble of Modified Base Model : Model 7
(Dropout 40%, 1 extra CNN Layer)

## Approach
- Create a List of Base Models
- Take all the Preprocessed Images from each Class
- Split Training data into Train and Validation with 0.7 traim-test ratio
- Perform this splitting for each model in a loop with a different random state so that shuffling of images is different for each model.
- Perform Augmentation on each class of training data splitted above 
- Train the models on augmented data validating on 30% validation data
- Once all models in list are trained, use max voting to find predction of validation image set

Before running make sure there is no augmented & test folder in the directory of notebook since it is created in this file itself. If it exists delete them. A caveat here is that if you wish to run Model_1_2_3_4.ipynb file again, you need to run Preprocessing Notebook again to form required folders.

Also, if required change the path seperator to run code smoothly on windows

In [1]:
# Importing the Libraries
# For missing libraries
# pip install -U package_name --user
from skimage import io
import os
import cv2
from tqdm import tqdm
from glob import glob
import albumentations
from albumentations import ShiftScaleRotate, HorizontalFlip, VerticalFlip, RandomBrightnessContrast
import warnings
import shutil
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from functools import partial
from sklearn.utils import shuffle
warnings.filterwarnings("ignore")

In [2]:
# Defining Augmentation Functions
# Shifting, Scaling & Rotations
def shiftscalerotate(images, save_path,
                     augment = True,
                     resize = [256,256],
                     random=1.0,
                     rotation=0,
                     shift=0,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     always=True,
                     call=1):
    H, W = resize
    for x in tqdm(images, total= len(images)):
        # First, we have to extract the image name, image extention.
        name = x.split("/")[-1].split(".")
        image_name = name[0]
        image_extn = name[1]

        # Now that we have the names, we have to read the image
        x = cv2.imread(x, cv2.IMREAD_COLOR)

        if augment ==True:
            aug = ShiftScaleRotate(p=random, rotate_limit=(-30,30), interpolation=interpolation,
                                  always_apply=always)
            augmented = aug(image=x)
            x1 = augmented["image"]
            save_images = [x, x1] # save original image and the augmented image
        # If the augment paramter is set to False, The function only saves the original image and mask to the defined save_path
        else:
            save_images = [x] 
        idx = 0
        for i in save_images:
            i = cv2.resize(i, (W,H))
            if idx == 0:
                tmp_img_name = f"{image_name}.{image_extn}"
            else:
                aug_ext = '_ssr'*call
                tmp_img_name = f"{image_name}{aug_ext}.{image_extn}"
            image_path = os.path.join(save_path, tmp_img_name)
            cv2.imwrite(image_path, i)
            idx +=1
# Horizontal Flipping
def horizontalflip(images, save_path,
                   augment = True,
                   resize = [256,256],
                   random=1.0,
                   always=True):
    H, W = resize
    for x in tqdm(images, total= len(images)):
        # First, we have to extract the image name, image extention.
        name = x.split("/")[-1].split(".")
        image_name = name[0]
        image_extn = name[1]
        # Now that we have the names, we have to read the image and the masks 
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        if augment ==True:
            aug = HorizontalFlip(always_apply=always, p=random)
            augmented = aug(image=x)
            x1 = augmented["image"]
            save_images = [x, x1] # make sure only x1. If x is given in 2nd augmentation onwards, then duplicate images will be saved
        # If the augment paramter is set to False, The function only saves the original image and mask to the defined save_path
        else:
            save_images = [x]
        idx = 0
        for i in save_images:
            i = cv2.resize(i, (W,H))
            if idx == 0:
                tmp_img_name = f"{image_name}.{image_extn}"
            else:
                tmp_img_name = f"{image_name}_hf.{image_extn}"
            image_path = os.path.join(save_path, tmp_img_name)
            cv2.imwrite(image_path, i)
            idx +=1
#vertical flipping
def verticalflip(images,
                 save_path,
                 augment = True,
                 resize=[256,256],
                 random=1.0,
                 always=True):
    H, W = resize
    for x in tqdm(images, total= len(images)):
        # First, we have to extract the image name, image extention.
        name = x.split("/")[-1].split(".")
        image_name = name[0]
        image_extn = name[1]
        # Now that we have the names, we have to read the image
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        if augment ==True:
            aug = VerticalFlip(always_apply=always, p=random)
            augmented = aug(image=x)
            x1 = augmented["image"]
            save_images = [x, x1]
        # If the augment paramter is set to False, The function only saves the original image and mask to the defined save_path
        else:
            save_images = [x]
        idx = 0
        for i in save_images:
            i = cv2.resize(i, (W,H))
            if idx == 0:
                tmp_img_name = f"{image_name}.{image_extn}"
            else:
                tmp_img_name = f"{image_name}_vf.{image_extn}"
            image_path = os.path.join(save_path, tmp_img_name)
            cv2.imwrite(image_path, i)
            idx +=1
def brightness_contrast(images,
                        save_path,
                        augment = True,
                        resize=[256,256],
                        random=1.0,
                        brightness=0.2, 
                        contrast=0.2,
                        by_max=False,
                        always=True):
    H, W = resize
    for x in tqdm(images, total= len(images)):
        # First, we have to extract the image name, image extention.
        name = x.split("/")[-1].split(".")
        image_name = name[0]
        image_extn = name[1]
        # Now that we have the names, we have to read the image
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        if augment ==True:
            aug = RandomBrightnessContrast(brightness_limit=brightness,
                                           contrast_limit=contrast,
                                           brightness_by_max=by_max,
                                           always_apply=always,
                                           p=random)
            augmented = aug(image=x)
            x1 = augmented["image"]
            save_images = [x, x1]
        # If the augment paramter is set to False, The function only saves the original image and mask to the defined save_path
        else:
            save_images = [x]
        idx = 0
        for i in save_images:
            i = cv2.resize(i, (W,H))
            if idx == 0:
                tmp_img_name = f"{image_name}.{image_extn}"
            else:
                tmp_img_name = f"{image_name}_bc.{image_extn}"
            image_path = os.path.join(save_path, tmp_img_name)
            cv2.imwrite(image_path, i)
            idx +=1

In [3]:
# Define ensemble of 10 models
n = 10
ensemble = [0]* n
def shuffling_tt(X, rs):
    X= shuffle(X, random_state=rs)
    return X

In [4]:
# Create & Complie Ensemble
DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu',
                        padding="SAME")
for k in range(n):
    ensemble[k] = keras.models.Sequential([
                    DefaultConv2D(filters=16, input_shape=[128, 128, 3]),
                    keras.layers.AveragePooling2D(pool_size=2),
                    DefaultConv2D(filters=32),
                    keras.layers.MaxPooling2D(pool_size=2),
                    DefaultConv2D(filters=64),
                    keras.layers.MaxPooling2D(pool_size=2),
                    DefaultConv2D(filters=128),
                    keras.layers.MaxPooling2D(pool_size=2),
                    keras.layers.Dropout(0.4),
                    keras.layers.Flatten(),
                    keras.layers.Dense(units=7, activation='softmax'),
                    ])
    ensemble[k].compile(loss='categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])

2022-03-11 02:42:21.549156: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-11 02:42:21.549341: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [5]:
def shuffling(X):
    X= shuffle(X, random_state=42)
    return X
out_path = 'augmented/'
# Performing Augmentations Class-Wise
# AKIEC
def augment_akiec(image_data):
    out_data = out_path+'akiec'
    shiftscalerotate(image_data,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0.0625,
                     scale=0.2,
                     interpolation=cv2.INTER_NEAREST)
    ssr_images = glob('augmented/akiec/*')
    horizontalflip(ssr_images, out_data,
                       augment = True,
                       resize = [128,128],
                       random=1.0)
    hf_images = glob('augmented/akiec/*')
    verticalflip(hf_images,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    df_images = glob('augmented/akiec/*')
    brightness_contrast(df_images,
                            out_data,
                            augment = True,
                            resize=[128,128],
                            random=1.0,
                            brightness=0.2, 
                            contrast=0.2,
                            by_max=False)
    # Selecting remaining images by shuffling all and selecting required
    bc_images = glob('augmented/akiec/*')
    # shuffle and take only required number of images to perform next augmentation:
    bc_images_shuf = shuffling(bc_images)
    target = 4693
    available = len(bc_images)
    bc_selected = bc_images_shuf[0:target-available]
    shiftscalerotate(bc_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=0,
                     shift=0,
                     scale=0.3,
                     interpolation=cv2.INTER_NEAREST,
                     call=2)

#---------------------------------------------------------------------------------------------
    # DF
def augment_df(image_data):
    out_data = out_path+'df'
    shiftscalerotate(image_data,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0.0625,
                     scale=0.2,
                     interpolation=cv2.INTER_NEAREST,
                     call=1)
    ssr_images = glob('augmented/df/*')
    horizontalflip(ssr_images, out_data,
                       augment = True,
                       resize = [128,128],
                       random=1.0)
    hf_images = glob('augmented/df/*')
    verticalflip(hf_images,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    df_images = glob('augmented/df/*')
    brightness_contrast(df_images,
                            out_data,
                            augment = True,
                            resize=[128,128],
                            random=1.0,
                            brightness=0.2, 
                            contrast=0.2,
                            by_max=False)
    bc_images = glob('augmented/df/*')
    shiftscalerotate(bc_images,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=2)
    r_images = glob('augmented/df/*')
    # shuffle and take only required number of images to perform next augmentation:
    r_images_shuf = shuffling(r_images)
    target = 4693
    available = len(r_images)
    r_selected = r_images_shuf[0:target-available]
    shiftscalerotate(r_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=80,
                     shift=0.2,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=3)
    
#---------------------------------------------------------------------------------------------
# VASC
def augment_vasc(image_data):
    out_data = out_path+'vasc'
    shiftscalerotate(image_data,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0.0625,
                     scale=0.2,
                     interpolation=cv2.INTER_NEAREST,
                     call=1)
    ssr_images = glob('augmented/vasc/*')
    horizontalflip(ssr_images, out_data,
                       augment = True,
                       resize = [128,128],
                       random=1.0)
    hf_images = glob('augmented/vasc/*')
    verticalflip(hf_images,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    df_images = glob('augmented/vasc/*')
    brightness_contrast(df_images,
                            out_data,
                            augment = True,
                            resize=[128,128],
                            random=1.0,
                            brightness=0.2, 
                            contrast=0.2,
                            by_max=False)
    bc_images = glob('augmented/vasc/*')
    shiftscalerotate(bc_images,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=2)
    r_images = glob('augmented/vasc/*')
    # shuffle and take only required number of images to perform next augmentation:
    r_images_shuf = shuffling(r_images)
    target = 4693
    available = len(r_images)
    r_selected = r_images_shuf[0:target-available]
    shiftscalerotate(r_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=80,
                     shift=0.2,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=3)

#---------------------------------------------------------------------------------------------
# BCC
def augment_bcc(image_data):
    out_data = out_path+'bcc'
    shiftscalerotate(image_data,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0.0625,
                     scale=0.2,
                     interpolation=cv2.INTER_NEAREST,
                     call=1)
    ssr_images = glob('augmented/bcc/*')
    horizontalflip(ssr_images, out_data,
                       augment = True,
                       resize = [128,128],
                       random=1.0)
    hf_images = glob('augmented/bcc/*')
    verticalflip(hf_images,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    r_images = glob('augmented/bcc/*')
    # shuffle and take only required number of images to perform next augmentation:
    r_images_shuf = shuffling(r_images)
    target = 4693
    available = len(r_images)
    r_selected = r_images_shuf[0:target-available]
    shiftscalerotate(r_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=80,
                     shift=0.2,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=2)

#---------------------------------------------------------------------------------------------
# BKL
def augment_bkl(image_data):
    out_data = out_path+'bkl'
    shiftscalerotate(image_data,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0.0625,
                     scale=0.2,
                     interpolation=cv2.INTER_NEAREST,
                     call=1)
    ssr_images = glob('augmented/bkl/*')
    verticalflip(ssr_images,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    r_images = glob('augmented/bkl/*')
    # shuffle and take only required number of images to perform next augmentation:
    r_images_shuf = shuffling(r_images)
    target = 4693
    available = len(r_images)
    r_selected = r_images_shuf[0:target-available]
    shiftscalerotate(r_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=80,
                     shift=0.2,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=2)
    
#---------------------------------------------------------------------------------------------
# MEL
def augment_mel(image_data):
    out_data = out_path+'mel'
    verticalflip(image_data,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    vf_images = glob('augmented/mel/*')
    brightness_contrast(vf_images,
                            out_data,
                            augment = True,
                            resize=[128,128],
                            random=1.0,
                            brightness=0.2, 
                            contrast=0.2,
                            by_max=False)
    r_images = glob('augmented/mel/*')
    # shuffle and take only required number of images to perform next augmentation:
    r_images_shuf = shuffling(r_images)
    target = 4693
    available = len(r_images)
    r_selected = r_images_shuf[0:target-available]
    shiftscalerotate(r_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=80,
                     shift=0.2,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=1)
    
#---------------------------------------------------------------------------------------------
# NV
# Selecting remaining images by shuffling all and selecting required
def augment_nv(image_data):
    for image in image_data:
        shutil.copyfile(image, f'augmented/nv/{image.split("/")[-1]}')

In [6]:
# Augment, Split and Train
classes = ['akiec','bcc','bkl','df','nv','mel','vasc']

TRAIN_DIR = 'Processed_Data/train'
model_train = [0] * n
epochs = 30
batch_size = 64
for j in range(n):
    # Train test Split
    rs = np.random.randint(100,150)*(j+1)
    for clas in classes:
        os.makedirs(f'augmented/{clas}')
    for cl in classes:
        r_images = glob(f'{TRAIN_DIR}/{cl}/*')
        r_images_shuf = shuffling_tt(r_images,rs)
        train = int(0.7*len(r_images))
        r_train = r_images_shuf[0:train]
        r_test = r_images_shuf[train:]
        if cl == 'akiec':
            augment_akiec(r_train)
        elif cl == 'bcc':
            augment_bcc(r_train)
        elif cl == 'bkl':
            augment_bkl(r_train)
        elif cl == 'df':
            augment_df(r_train)
        elif cl == 'mel':
            augment_mel(r_train)
        elif cl == 'vasc':
            augment_vasc(r_train)
        else:
            augment_nv(r_train)
        if not os.path.isdir(f'test/{cl}'):
            os.makedirs(f'test/{cl}')
        for image in r_test:
            shutil.copyfile(image, f'test/{cl}/{image.split("/")[-1]}')
    # Model Training

    train_dir = 'augmented'
    test_dir = 'test'
    datagen = ImageDataGenerator(rescale = 1./255)

    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical') # set as training data

    validation_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
    model_train[j] = ensemble[j].fit_generator(train_generator,
                                               steps_per_epoch=train_generator.samples//batch_size,
                                               validation_steps=validation_generator.samples//batch_size,
                                               validation_data = validation_generator,
                                               epochs = epochs)
    print("CNN Model {0:d}: Epochs={1:d}, Training accuracy={2:.5f}, Validation accuracy={3:.5f}".
          format(j+1,epochs,
                 max(model_train[j].history['accuracy']),
                 max(model_train[j].history['val_accuracy']) ))
    shutil.rmtree('augmented')
    shutil.rmtree('test')

100%|█████████████████████████████████████| 1525/1525 [00:01<00:00, 1510.25it/s]


Found 32740 images belonging to 7 classes.
Found 3006 images belonging to 7 classes.
Epoch 1/30


2022-03-11 02:43:08.529329: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-11 02:43:08.768602: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - ETA: 0s - loss: 1.3561 - accuracy: 0.4660

2022-03-11 02:43:47.192907: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

Found 32743 images belonging to 7 classes.
Found 3006 images belonging to 7 classes.
Epoch 1/30


2022-03-11 06:43:16.205541: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - ETA: 0s - loss: 1.3476 - accuracy: 0.4641

2022-03-11 06:43:55.845641: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - 53s 102ms/step - loss: 1.3476 - accuracy: 0.4641 - val_loss: 1.0276 - val_accuracy: 0.6087
Epoch 2/30
511/511 [==============================] - 51s 100ms/step - loss: 1.0891 - accuracy: 0.5800 - val_loss: 0.8857 - val_accuracy: 0.6624
Epoch 3/30
511/511 [==============================] - 52s 101ms/step - loss: 0.9705 - accuracy: 0.6280 - val_loss: 0.9422 - val_accuracy: 0.6369
Epoch 4/30
511/511 [==============================] - 52s 101ms/step - loss: 0.8611 - accuracy: 0.6762 - val_loss: 0.9283 - val_accuracy: 0.6450
Epoch 5/30
511/511 [==============================] - 52s 102ms/step - loss: 0.7951 - accuracy: 0.7001 - val_loss: 0.9132 - val_accuracy: 0.6440
Epoch 6/30
511/511 [==============================] - 52s 101ms/step - loss: 0.7315 - accuracy: 0.7271 - val_loss: 0.8169 - val_accuracy: 0.6838
Epoch 7/30
511/511 [==============================] - 52s 101ms/step - loss: 0.6798 - accuracy: 0.7465 - val_loss: 0.9294 - val_accuracy: 0.6

In [8]:
#Confution Matrix and Classification Report
#Result
results = np.zeros( (validation_generator.samples,7) ) 
results

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
validation_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

Found 3007 images belonging to 7 classes.


In [9]:
for j in range(n):
    results = results + ensemble[j].predict_generator(validation_generator,
                                                      validation_generator.samples // batch_size+1)

2022-03-11 07:42:44.493517: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 07:42:57.326416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 07:43:10.082066: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 07:43:22.875017: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 07:43:35.685206: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 07:43:48.512547: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 07:44:01.322831: I tensorflow/core/grappler/optimizers/cust

In [10]:
results = np.argmax(results,axis = 1)
results

array([0, 0, 0, ..., 6, 6, 6])

In [12]:

# Y_pred = model.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
# y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, results))
print('Classification Report')
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
print(classification_report(validation_generator.classes, results, target_names=target_names))

Confusion Matrix
[[  98    0    0    0    1    0    0]
 [   1  151    1    0    0    2    0]
 [   5    6  301    0    3   14    0]
 [   0    0    0   35    0    0    0]
 [   1    0    2    0  323    8    0]
 [   7   13   36    5   30 1921    0]
 [   0    0    0    0    0    0   43]]
Classification Report
              precision    recall  f1-score   support

       akiec       0.88      0.99      0.93        99
         bcc       0.89      0.97      0.93       155
         bkl       0.89      0.91      0.90       329
          df       0.88      1.00      0.93        35
         mel       0.90      0.97      0.93       334
          nv       0.99      0.95      0.97      2012
        vasc       1.00      1.00      1.00        43

    accuracy                           0.96      3007
   macro avg       0.92      0.97      0.94      3007
weighted avg       0.96      0.96      0.96      3007



In [15]:
# Saving Model in Keras Default Format
for k in range(n):
    ensemble[k].save(f'Ensemble3_model{k}')
# Saving Model in H5 Format
for k in range(n):
    ensemble[k].save(f'Ensemble3_model{k}.h5')

2022-03-12 15:33:07.916761: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: Ensemble3_model0/assets
INFO:tensorflow:Assets written to: Ensemble3_model1/assets
INFO:tensorflow:Assets written to: Ensemble3_model2/assets
INFO:tensorflow:Assets written to: Ensemble3_model3/assets
INFO:tensorflow:Assets written to: Ensemble3_model4/assets
INFO:tensorflow:Assets written to: Ensemble3_model5/assets
INFO:tensorflow:Assets written to: Ensemble3_model6/assets
INFO:tensorflow:Assets written to: Ensemble3_model7/assets
INFO:tensorflow:Assets written to: Ensemble3_model8/assets
INFO:tensorflow:Assets written to: Ensemble3_model9/assets


## Loading Saved Models

In [ ]:
# To Load the model
# It can be used to reconstruct the model identically.
# reconstructed_ensemble = [0]*n
# for k in range(n):
#     reconstructed_ensemble[k] = keras.models.load_model(f"Saved Models/Ensemble3/Ensemble3_model{k}")